<a href="https://colab.research.google.com/github/Jasur-cmd/Jasur-cmd.github.io/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22ChatBot_Day_3_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install python-telegram-bot --upgrade

In [2]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import logging
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [3]:
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Привет!', 'Здарова', 'Хей-хей!!'],
#             'responses': ['Хай', 'Добрый вечер!', 'Здравствуйте!']
#         },
#         'bye': {
#             'examples': ['Пока', 'Увидимся!', 'Покеда'],
#             'responses': ['До свидания', 'Прощайте', 'Сайонара!']
#         }
#     }
# }
with open('/content/BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)

# День 1

In [4]:
def clean(text):
  text = text.lower()
  cleaned_text = ''
  for ch in text:
    if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
      cleaned_text = cleaned_text + ch
  return cleaned_text

def get_intent(text):
  for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
      w1 = clean(example)
      w2 = clean(text)
      if nltk.edit_distance(w1, w2) / max(len(w1), len(w2)) < 0.4:
        return intent
  return 'интент не найден'

# День 2

In [5]:
X = []
y = []

for intent in BOT_CONFIG['intents'].keys():
    try:
        for example in BOT_CONFIG['intents'][intent]['examples']:
            X.append(example)
            y.append(intent)
    except:
        pass

len(X), len(y), len(set(y))

(841, 841, 243)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
len(X_train), len(X_test)

(672, 169)

In [7]:
vectorizer = CountVectorizer(preprocessor=clean, analyzer='char', ngram_range=(2,3))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

len(vectorizer.get_feature_names())

2476

In [8]:
log_reg = LogisticRegression(C=0.2)
log_reg.fit(X_train_vect, y_train)
log_reg.score(X_train_vect, y_train)

0.8065476190476191

In [9]:
log_reg.score(X_test_vect, y_test)

0.1834319526627219

In [10]:
def get_intent_by_model(text):
    return log_reg.predict(vectorizer.transform([text]))[0]

In [11]:
def bot(question):
  intent = get_intent_by_model(question)
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [12]:
# question = ''
# while True:
#   question = input()
#   if question != 'стоп':
#       answer = bot(question)
#       print(answer)
#   else:
#       break

# День 3

In [13]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    question = update.message.text
    try:
      answer = bot(question)
    except:
      answer = 'Извините, что-то сломалось =('
      
    update.message.reply_text(answer)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1889922408:AAF_EwPtHvz0TYnYuTom6WCSfcOaWfFPEbw")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()

2021-09-15 18:43:34,652 - apscheduler.scheduler - INFO - Scheduler started
